<a href="https://colab.research.google.com/github/fdac25/brawlhalla/blob/main/usage_over_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
import plotly.express as px
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# data = pd.read_csv("brawlhalla_matches.csv")
data = pd.read_csv("brawlhalla_tournament_match_data.csv")
data['date'] = pd.to_datetime(data['date'])
data

# Getting Complete List Of All Legends Used

In [ ]:
legends = []
for row in data.itertuples(index=True):
  p1legends = list(filter(bool, [string.strip() for string in row.player1_legends.split(",")])) # Grab P1's legends and remove whitespace/empties
  for legend in p1legends: # Iterate through all of P1's legends
    if legend not in legends: # If legend is not already recorded in legends
        legends.append(legend) # Append legend to list

  p2legends = list(filter(bool, [string.strip() for string in row.player2_legends.split(",")])) # Grab P2's legends and remove whitespace/empties
  for legend in p2legends: # Iterate through all of P2's legends
    if legend not in legends: # If legend is not already recorded in legends
        legends.append(legend) # Append legend to list

if "Not Reported" in legends:
  legends.remove("Not Reported") # Remove 'Not Reported' from the list of legends

legends

# Getting Total Matches Where Legends Were Reported

In [ ]:
notReported = "Not Reported"
totalMatches = 0

for row in data.itertuples(index=True):
  #print(f"{row.player1_legends}\t{row.player2_legends}")
  if notReported not in row.player1_legends and notReported not in row.player2_legends:
    totalMatches += 2 # Adding 2 instead of 1 due to the fact that either player could choose or not choose a given character

print(totalMatches)

In [ ]:
matchesPerDay = data.groupby('date').size().reset_index(name='count')
matchesPerDay

In [ ]:
matchesDict = dict(zip(matchesPerDay['date'], matchesPerDay['count']))
# matchesDict = {str(key): value for key, value in matchesDict.items()}
matchesDict

# Calculating Win/Usage Of Each Character

In [ ]:
temp = list(data['year'].unique())
temp.sort()
years = ['ALL'] + temp

@interact(
    legend=legends,
    fill=[True,False],
    year=years
)
def plot_legend(legend=legends[0], fill=True, year=years[0]):
    matches = data

    if year != 'ALL':
      matches = matches[matches['year'] == year]

    uses = []

    for row in matches.itertuples(index=True):
      # Record winner of the match as an integer
      if "1" in row.winner:
        winner = 1
      elif "2" in row.winner:
        winner = 2

      p1legends = list(set(list(filter(bool, [string.strip() for string in row.player1_legends.split(",")])))) # Grab P1's legends and remove whitespace/empties/duplicates
      if legend in p1legends:
        uses.append([row.match_id, row.date, legend, winner == 1])

      p2legends = list(set(list(filter(bool, [string.strip() for string in row.player2_legends.split(",")])))) # Grab P2's legends and remove whitespace/empties/duplicates
      if legend in p2legends:
        uses.append([row.match_id, row.date, legend, winner == 2])

    usesdf = pd.DataFrame(uses, columns=['match_id', 'datetime', 'legend', 'won']) # Convert list of lists to dataframe
    usesdf['datetime'] = pd.to_datetime(usesdf['datetime']) # Convert datetime column to actual datetime
    usesdf.sort_values(by='datetime', inplace=True) # Sort by datetime
    usesdf.reset_index(drop=True, inplace=True) # Reset indices

    daily_counts = usesdf.groupby('datetime').size().reset_index(name='count')
    daily_counts = daily_counts.set_index('datetime')

    if fill:
      full_date_range = pd.date_range(start=daily_counts.index.min(), end=daily_counts.index.max(), freq='D')
      daily_counts = daily_counts.reindex(full_date_range).fillna(0)

    plt.figure(figsize=(15,5))
    # monthly_counts = all_counts[all_counts['project']==project]
    plt.plot(daily_counts.index, daily_counts["count"], marker="o", linestyle="-")
    plt.xlabel("Date")
    plt.ylabel("Uses")
    plt.title(legend)
    plt.grid(True)
    plt.tight_layout()
    ax = plt.gca()
    for label in ax.xaxis.get_ticklabels()[::2]:
        label.set_visible(False)
    plt.xticks(rotation=45, fontsize=7)
    plt.show()

In [ ]:
@interact(
    legend=legends,
    fill=[True,False],
    year=years
)
def plot_legend(legend=legends[0], fill=True, year=years[0]):
    matches = data

    if year != 'ALL':
      matches = matches[matches['year'] == year]

    uses = []

    for row in matches.itertuples(index=True):
      # Record winner of the match as an integer
      if "1" in row.winner:
        winner = 1
      elif "2" in row.winner:
        winner = 2

      p1legends = list(set(list(filter(bool, [string.strip() for string in row.player1_legends.split(",")])))) # Grab P1's legends and remove whitespace/empties/duplicates
      if legend in p1legends:
        uses.append([row.match_id, row.date, legend, winner == 1])

      p2legends = list(set(list(filter(bool, [string.strip() for string in row.player2_legends.split(",")])))) # Grab P2's legends and remove whitespace/empties/duplicates
      if legend in p2legends:
        uses.append([row.match_id, row.date, legend, winner == 2])

    usesdf = pd.DataFrame(uses, columns=['match_id', 'datetime', 'legend', 'won']) # Convert list of lists to dataframe
    usesdf['datetime'] = pd.to_datetime(usesdf['datetime']) # Convert datetime column to actual datetime
    usesdf.sort_values(by='datetime', inplace=True) # Sort by datetime
    usesdf.reset_index(drop=True, inplace=True) # Reset indices

    daily_counts = usesdf.groupby('datetime').size().reset_index(name='count')

    # for i in range(len(daily_counts)):
    #   daily_counts.iloc[i]['usage%'] = daily_counts.iloc[i]['count'] / matchesPerDay.loc[matchesPerDay['date'] == daily_counts.iloc[i]['datetime'], 'count'].iloc[0]

    # daily_counts['usage'] = daily_counts['count'] / matchesPerDay.loc[matchesPerDay['date'] == daily_counts['datetime'], 'count'].iloc[0] * 100


    daily_counts = daily_counts.set_index('datetime')

    if fill:
      full_date_range = pd.date_range(start=daily_counts.index.min(), end=daily_counts.index.max(), freq='D')
      daily_counts = daily_counts.reindex(full_date_range).fillna(0)

    # for row in daily_counts.itertuples(index=True):
    #   row['usage'] = row['count'] / matchesPerDay.loc[matchesPerDay['date'] == row.Index, 'count'].iloc[0] * 100

    # for i in range(len(daily_counts)):
    #   daily_counts.iloc[i]['usage'] = daily_counts.iloc[i]['count'] / matchesPerDay.loc[matchesPerDay['date'] == daily_counts.iloc[i].index, 'count'].iloc[0]

    # daily_counts['usage%'] = daily_counts['count'] / matchesPerDay.loc[matchesPerDay['date'] == daily_counts.index[0], 'count'].iloc[0] * 100
    # daily_counts['usage%'] = daily_counts['count'] / matchesDict[f'{daily_counts.index}'] * 100

    # daily_counts['usage%'] = daily_counts.apply(
    #     lambda row: (row['count'] / matchesDict.get(str(row.name.date()), 1) * 100) if matchesDict.get(str(row.name.date()), 0) > 0 else 0,
    #     axis=1
    # )

    # daily_counts['usage'] = None

    # for idx in daily_counts.index:
    #     value = daily_counts.loc[idx, 'count'] / matchesDict[str(idx)]
    #     daily_counts.at[idx, 'usage'] = value

    daily_counts['usage%'] = daily_counts.apply(
        lambda row: (row['count'] / matchesDict.get(row.name, 1) * 100)
        if matchesDict.get(row.name, 0) > 0 else 0,
        axis=1
    )

    plt.figure(figsize=(15,5))
    # plt.plot(daily_counts.index, daily_counts["count"], marker="o", linestyle="-")
    plt.plot(daily_counts.index, daily_counts["usage%"], marker="o", linestyle="-")
    plt.xlabel("Date")
    plt.ylabel("Usage %")
    plt.title(legend)
    plt.grid(True)
    plt.tight_layout()
    ax = plt.gca()
    for label in ax.xaxis.get_ticklabels()[::2]:
        label.set_visible(False)
    plt.xticks(rotation=45, fontsize=7)
    plt.show()